In [1]:
 !pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 21.6 MB 1.7 MB/s 


In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import copy
import cv2
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/Data/features_30_sec.csv")
df = df[['filename','label']]

In [6]:
df = df[df['filename'] != "jazz.00054.wav"]

In [7]:
df = df.reset_index()

In [8]:
df.pop('index')

0        0
1        1
2        2
3        3
4        4
      ... 
994    995
995    996
996    997
997    998
998    999
Name: index, Length: 999, dtype: int64

In [9]:
class_name = {}
n = 0
for i in set(df['label']):
    class_name[i] = n
    n+=1
num_classes = n

In [10]:
class_name

{'blues': 3,
 'classical': 5,
 'country': 9,
 'disco': 8,
 'hiphop': 1,
 'jazz': 6,
 'metal': 4,
 'pop': 0,
 'reggae': 2,
 'rock': 7}

In [11]:
df['label'] = df['label'].map(class_name)

In [12]:
for i in range(len(df)):
    temp = df['filename'][i].split(".")
    df['filename'][i] = "Data/images_original/" + temp[0] + "/" + temp[0] + temp[1] + ".png"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
train, test = train_test_split(df, test_size=0.30, random_state=42, stratify = df['label'])
test, val = train_test_split(test, test_size=0.50, random_state=42, stratify = test['label'])

In [14]:
dataset_sizes = {'train': len(train), 'test': len(test), 'val': len(val)}

In [15]:
class GenreDataset(Dataset):
    """Genre dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.csv.iloc[idx, 0]
        image = cv2.imread("/content/drive/MyDrive/" + img_name,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = np.expand_dims(image, axis=-1)
        details = self.csv.iloc[idx, 1:]
        sample = {'image': image, 'label': details[0]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [16]:
class PreProcessing(object):

    def __init__(self):
        pass

    def __call__(self, sample):
        image, turtle_id = sample['image'], sample['label']
#         h, w = image.shape[:2]
        
        ### ADD PREPROCESSING CODE HERE
#         image = np.array([image])
        
        return [np.transpose(image, (2, 1, 0)), turtle_id]

In [17]:
train_transformed_dataset = GenreDataset(csv_file=train,
                                               transform=transforms.Compose([
                                               PreProcessing()
                                           ]))
test_transformed_dataset = GenreDataset(csv_file=test,
                                               transform=transforms.Compose([
                                               PreProcessing()
                                           ]))
val_transformed_dataset = GenreDataset(csv_file=val,
                                               transform=transforms.Compose([
                                               PreProcessing()
                                           ]))

In [18]:
dataloaders = {'train' : DataLoader(train_transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=0),
              'test' : DataLoader(test_transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=0),
              'val' : DataLoader(val_transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=0)}

In [19]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid((images[0].detach()[:nmax]), nrow=8).permute(1, 2, 0))
def show_batch(dl, nmax=64):
    for images in dl:
        print(images[1].shape)
        show_images(images, nmax)
        break

In [20]:
def show_pics(image):
    print(image.shape)
    plt.imshow(image)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [21]:
next(iter(train_transformed_dataset[0])).shape

(1, 339, 221)

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.type(torch.DoubleTensor)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Need to define our model below

In [24]:
import torch
import torch.nn as nn

def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

def conv_layer(ni, nf, ks=3, stride=1,act=True):
    bn = nn.BatchNorm2d(nf)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    act_fn = nn.ReLU(inplace=True)
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf, nf)
        self.conv2 = conv_layer(nf, nf)
    def forward(self, x):
        return x + self.conv2(self.conv1(x))

def conv_layer_averpl(ni, nf):
    aver_pl = nn.AvgPool2d(kernel_size=2, stride=2)
    return nn.Sequential(conv_layer(ni, nf), aver_pl)

model_ft = nn.Sequential(
    conv_layer_averpl(1, 64),
    ResBlock(64),
    conv_layer_averpl(64, 64),
    ResBlock(64),
    conv_layer_averpl(64, 128),
    ResBlock(128),
    conv_layer_averpl(128, 256),
    ResBlock(256),
    conv_layer_averpl(256, 512),
    ResBlock(512),
    nn.AdaptiveAvgPool2d((2,2)),
    nn.Flatten(),
    nn.Linear(2048, 40),
    nn.Linear(40, 10)
)

In [25]:
# model_ft = models.resnet18(pretrained=True) ### Can change this to try out other models available 
# num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [26]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                           num_epochs=5)
# torch.save(model_ft.state_dict(),"weights.pth")

Epoch 0/4
----------
train Loss: 2.1382 Acc: 0.2332
val Loss: 1.8259 Acc: 0.3600

Epoch 1/4
----------
train Loss: 1.7428 Acc: 0.3619
val Loss: 1.5366 Acc: 0.3667

Epoch 2/4
----------
train Loss: 1.6028 Acc: 0.4149
val Loss: 1.7742 Acc: 0.4600

Epoch 3/4
----------
train Loss: 1.5074 Acc: 0.4506
val Loss: 2.3896 Acc: 0.3133

Epoch 4/4
----------
train Loss: 1.4447 Acc: 0.4864
val Loss: 2.0504 Acc: 0.4400

Training complete in 8m 2s
Best val Acc: 0.460000


## The code block below takes the model trained, extracts the song embedding and does similarity scoring

In [97]:
my_model = nn.Sequential(*list(model_ft.modules())[0][:-1]) # strips off last linear layer
class SongEncoderDataset(Dataset):
    """Genre dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.csv.iloc[idx, 0]
        image = cv2.imread("/content/drive/MyDrive/" + img_name,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = np.expand_dims(image, axis=-1)
        details = self.csv.iloc[idx, 1:]
        sample = {'image': image, 'label': img_name.split('/')[-1].split(".")[0]}

        if self.transform:
            sample = self.transform(sample)

        return sample

transformed_dataset = SongEncoderDataset(csv_file=df,
                                               transform=transforms.Compose([
                                               PreProcessing()
                                           ]))
whole_dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=False, num_workers=0)

def song_encoder(model):
                
    model.eval()   # Set model to evaluate mode

    song_name = []
    song_encoded = []

    # Iterate over data.
    for inputs, labels in whole_dataloader:
        inputs = inputs.to(device)

        outputs = model(inputs.float())
        temp = outputs.cpu().detach().numpy()
        song_name += list(labels)
        for i in temp:
          song_encoded.append(i)

    return song_name, song_encoded

song_name, song_encoded = song_encoder(my_model)
song_encoded = np.array(song_encoded)
import pickle
with open('song_embed.pkl', 'wb') as f:
   pickle.dump(song_encoded, f)
with open('song_name.pkl', 'wb') as f:
   pickle.dump(song_name, f)

In [98]:
import pickle
with open('song_embed.pkl', 'rb') as f:
   mynewlist = pickle.load(f)
with open('song_name.pkl', 'rb') as f:
   song_name = pickle.load(f)

## Song embedding and similarity stops here

In [99]:
def songRecomendation(song_name, song_embed, new_song, k=5):
  ls = np.argpartition(np.dot(song_embed,new_song), -k)[-k:]
  return [song_name[i] for i in ls]
songRecomendation(song_name, mynewlist, mynewlist[0])

['jazz00006', 'blues00031', 'blues00036', 'jazz00004', 'jazz00026']

In [ ]:
torch.save(model_ft.state_dict(),"weights.pth")

In [ ]:
import torch
import torch.nn as nn

def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

def conv_layer(ni, nf, ks=3, stride=1,act=True):
    bn = nn.BatchNorm2d(nf)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    act_fn = nn.ReLU(inplace=True)
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf, nf)
        self.conv2 = conv_layer(nf, nf)
    def forward(self, x):
        return x + self.conv2(self.conv1(x))

def conv_layer_averpl(ni, nf):
    aver_pl = nn.AvgPool2d(kernel_size=2, stride=2)
    return nn.Sequential(conv_layer(ni, nf), aver_pl)

model_ft = nn.Sequential(
    conv_layer_averpl(1, 64),
    ResBlock(64),
    conv_layer_averpl(64, 64),
    ResBlock(64),
    conv_layer_averpl(64, 128),
    ResBlock(128),
    conv_layer_averpl(128, 256),
    ResBlock(256),
    conv_layer_averpl(256, 512),
    ResBlock(512),
    nn.AdaptiveAvgPool2d((2,2)),
    nn.Flatten(),
    nn.Linear(2048, 40),
    nn.Linear(40, 10)
)

In [ ]:
model_ft.load_state_dict(torch.load("weights.pth"))
model_ft = model_ft.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def test_model(model, criterion, optimizer, scheduler):

    model.eval()   # Set model to evaluate mode

    t_output = []
    t_pred = []
    y_test = []
    top_k = []
    # Iterate over data.
    i = 1
    for inputs, labels in dataloaders['test']:
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        y_test.append(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(True):
            outputs = model(inputs.float())
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            t_output.append(outputs)
            t_pred.append(preds)
            temp1, temp2 = outputs.topk(5)
            top_k.append(temp2)

    y_test = torch.cat(y_test).cpu().detach().numpy() 
    y_test_num = torch.cat(t_pred).cpu().detach().numpy() 
    y_pred = torch.cat(top_k).cpu().detach().numpy() 
    print('\nConfusion Matrix')
    conf_mt = confusion_matrix(y_test_num, y_test)
    print(conf_mt)
    plt.matshow(conf_mt)
    plt.show()
    print('\nClassification Report')
    print(classification_report(y_test_num, y_test))

In [ ]:
test_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

RuntimeError: CUDA out of memory. Tried to allocate 74.00 MiB (GPU 0; 8.00 GiB total capacity; 6.34 GiB already allocated; 52.79 MiB free; 6.46 GiB reserved in total by PyTorch)